## Notebook Dashboard
Example of a fairly complex dashboard. Initially inspired by a Dash tutorial example

### Imports

In [1]:
# ipyplotly
from ipyplotly.datatypes import Figure
from ipyplotly.callbacks import Points, InputState

# pandas
import pandas as pd
from pandas.api.types import is_numeric_dtype

# numpy
import numpy as np

# ipywidgets
from ipywidgets import Dropdown, HBox, VBox

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/mtcars.csv')

In [3]:
numeric_cols = [col for col in df.columns if is_numeric_dtype(df[col])]
numeric_cols

['mpg', 'cyl', 'disp', 'hp', 'drat', 'wt', 'qsec', 'vs', 'am', 'gear', 'carb']

In [4]:
f = Figure()
f

Figure()

In [5]:
bar = f.add_bar(y=df.manufacturer.values, orientation='h')

In [6]:
f.layout.margin.l = 120
bar.marker.showscale = True
bar.marker.colorscale = 'viridis'

f.layout.width = 1100
f.layout.height = 800
bar.marker.line.width = 1
bar.marker.line.color = 'darkgray'

In [7]:
trace, points, state = bar, Points(), InputState()

# Bar click callback
def update_click(trace, points, state):
    new_clr = np.zeros(df['mpg'].size)
    new_clr[points.point_inds] = 1
    
    bar_line_sizes = np.ones(df['mpg'].size)
    bar_line_sizes[points.point_inds] = 3
    
    # Update pct line color
    par.line.color = new_clr
    
    # Update bar line color and width
    with f.batch_update():
        bar.marker.line.width = bar_line_sizes 
        bar.marker.line.color = new_clr

bar.on_click(update_click)
bar.on_selected(update_click)

In [8]:
f2 = Figure(layout={'width': 1100})
f2

Figure()

In [9]:
par = f2.add_parcoords(dimensions=[{
    'values': df[col].values, 
    'label': col,
    'range': [np.floor(df[col].min()), np.ceil(df[col].max())]} for col in numeric_cols])

In [10]:
# Set up selection colormap
par.line.colorscale = [[0, 'darkgray'], [1, 'red']]
par.line.cmin = 0
par.line.cmax = 1
par.line.color = np.zeros(df['mpg'].size)

bar.marker.line.colorscale = par.line.colorscale
bar.marker.line.cmin = 0
bar.marker.line.cmax = 1

In [11]:
# Widgets
dd = Dropdown(options=df.columns, description='X', value='mpg')
clr_dd = Dropdown(options=numeric_cols, description='Color')

def update_col(val):
    col = dd.value
    clr = clr_dd.value
    with f.batch_update():
        bar.x = df[col].values
        bar.marker.color = df[clr].values
        bar.marker.colorbar.title = clr
        f.layout.xaxis.title = col

dd.observe(update_col, 'value')
clr_dd.observe(update_col, 'value')

update_col(None)

## Display Dashboard
 - Dropdowns control barchart x-axis feature and coloring feature
 - Click or select bars to highlight in barchart and parallel coordinate diagram

In [12]:
VBox([f, HBox([dd, clr_dd]), f2])

VBox(children=(Figure(), HBox(children=(Dropdown(description='X', index=1, options=('manufacturer', 'mpg', 'cyl', 'disp', 'hp', 'drat', 'wt', 'qsec', 'vs', 'am', 'gear', 'carb'), value='mpg'), Dropdown(description='Color', options=('mpg', 'cyl', 'disp', 'hp', 'drat', 'wt', 'qsec', 'vs', 'am', 'gear', 'carb'), value='mpg'))), Figure()))

In [13]:
# Adjust barchart height
f.layout.height = 650

In [16]:
bar.unselected.marker.color = 'green'

In [18]:
bar.selected.marker.color = 'blue'